In [1]:
import tqdm

train = '''5764801
17807724'''


test = '''9232416
14144084'''


class Handshaker:
    
    def __init__(self, public_keys):
    
        self.public_key = {
            k: int(v)
            for k, v
            in zip(['card', 'door'], public_keys.split('\n'))} 
                
        self.loop_size = {}

    def transform_slow(self, subject, loop_size):
        
        for _loop_size in range(loop_size):
            self.transform_fast(subject=subject, loop_size=_loop_size)
            
        return self.transform_fast(subject=subject, loop_size=loop_size)
        
    @functools.lru_cache()
    def transform_fast(self, subject, loop_size):

        if loop_size == 0:
            return 1
        
        value = self.transform_fast(
            subject   = subject,
            loop_size = loop_size - 1)

        value *= subject
        value  = value % 20201227

        return value
    
    def set_loop_size(self, device_type):
        
        guess = 1
        while not self.is_valid_loop_size(device_type=device_type, guess=guess):
            guess += 1
        self.loop_size[device_type] = guess
        
        return self
    
    def is_valid_loop_size(self, device_type, guess):
        value = self.transform_fast(subject=7, loop_size=guess)
        return value == self.public_key[device_type]
    
    def get_encryption_key(self):
    
        encryption_key_1 = self.transform_slow(
            subject   = self.public_key['door'],
            loop_size = self.loop_size['card'])
                
        encryption_key_1 = self.transform_slow(
            subject   = self.public_key['card'],
            loop_size = self.loop_size['door'])
                
        assert encryption_key_1 == encryption_key_1
        
        return encryption_key_1

self = Handshaker(public_keys=train)
assert self.set_loop_size('card').set_loop_size('door').get_encryption_key() == 14897079

self = Handshaker(public_keys=test)
self.set_loop_size('card').set_loop_size('door').get_encryption_key()

1478097